In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignPlanar({}, overwrite_enabled=True)
gui = MetalGUI(design)
design.chips.main.size['size_x']='3mm'
design.chips.main.size['size_y']='3mm'

In [3]:
design.chips

{'main': {'material': 'silicon',
  'layer_start': '0',
  'layer_end': '2048',
  'size': {'center_x': '0.0mm',
   'center_y': '0.0mm',
   'center_z': '0.0mm',
   'size_x': '3mm',
   'size_y': '3mm',
   'size_z': '-750um',
   'sample_holder_top': '890um',
   'sample_holder_bottom': '1650um'}}}

In [24]:
q0

[name:    Q0
 class:   TransmonPocket6       
 options: 
   'pos_x'             : '-300um',                     
   'pos_y'             : '-500um',                     
   'orientation'       : '0.0',                        
   'chip'              : 'main',                       
   'layer'             : '1',                          
   'connection_pads'   : {
                         },
   'pad_gap'           : '20um',                       
   'inductor_width'    : '20um',                       
   'pad_width'         : '420um',                      
   'pad_height'        : '90um',                       
   'pocket_width'      : '650um',                      
   'pocket_height'     : '650um',                      
   'hfss_wire_bonds'   : False,                        
   'q3d_wire_bonds'    : False,                        
   'hfss_inductance'   : '10nH',                       
   'hfss_capacitance'  : 0,                            
   'hfss_resistance'   : 0,                     

In [56]:

cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-300um',
                                            pos_y='-500um',
                                            chip='main',
                                            pad_gap='20um',
                                            pad_width='440um',
                                            pad_height='70um'))]
                                            
qubits=q0
q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-160}um",
                                              pos_y=f"{q0y+112}um",
                                              pad_width="160um",
                                              pad_height="50um",
                                              cpw_stub_height="200um",
                                              chip="main"))
design.rebuild()
gui.rebuild()
gui.autoscale()

In [13]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="main",
                                         pos_x = '-700um',
                                         pos_y = '800um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="main",
                                         pos_x = '700um',
                                         pos_y = '800um',
                                         orientation='180',
                                         lead_length = '30um'))                                        

In [23]:
stg=ShortToGround(design, 'stg', options=Dict(chip="main",
                                                pos_x='0.64mm',
                                                pos_y='-0.2mm',
                                                orientation='270'))

In [15]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="main", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [16]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="main",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Resonator length

In [17]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [18]:
resonator_length_4=[]
a=find_resonator_length(7.5,10,6,4)
a=a.strip('mm')
a=float(a)-0.25 # subtract coupler length
resonator_length_4.append(a)
print(resonator_length_4)


[3.807562247348839]


In [45]:
jogs1=OrderedDict()
jogs1[0]=['R','300um']
jogs1[1]=['R', '300um']
jogs1[2]=['L','800um']
jogs1[3]=['R','200um']
jogs1[4]=['R','615um']
jogs1[5]=['L','200um']
jogs1[6]=['L','615um']


In [46]:
rr0=connect('rr0', 'c0', 'in','stg', 'short',
            resonator_length_4[0],jogs1,jogs, 
            '100um','0um', '947um', '100um')
            

# EPR Analysis

In [57]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [58]:
hfss.start()

INFO 04:02PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:02PM [load_ansys_project]: 	Opened Ansys App
INFO 04:02PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 04:02PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   KPS_Flipchip Bump
INFO 04:02PM [connect_design]: 	Opened active design
	Design:    Planar_Final2 [Solution type: Eigenmode]
INFO 04:02PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 04:02PM [connect]: 	Connected to project "KPS_Flipchip Bump" and design "Planar_Final2" 😀 



True

In [59]:
hfss.activate_ansys_design("Planar_Final3", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

04:02PM 27s WARNING [activate_ansys_design]: The design_name=Planar_Final3 was not in active project.  Designs in active project are: 
['flip6um_BUMP', 'flip8um_V6', 'flip8um_V6_BUMP', 'flip6um_V6', 'flip9um_V6', 'Planar-9', 'flipchip7um_Planar_again', 'Design_q3d', 'flip7um_BUMP', 'flip7um_V6', 'flip5um_V6', 'Planar_Final', 'Planar_Final2'].  A new design will be added to the project.  
INFO 04:02PM [connect_design]: 	Opened active design
	Design:    Planar_Final3 [Solution type: Eigenmode]
WARNING 04:02PM [connect_setup]: 	No design setup detected.
WARNING 04:02PM [connect_setup]: 	Creating eigenmode default setup.
INFO 04:02PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [64]:
hfss.close()


True

# LOM Analysis

In [60]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [61]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='6.01',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '6.01',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [62]:
c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 04:07PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:07PM [load_ansys_project]: 	Opened Ansys App
INFO 04:07PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 04:07PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   KPS_Flipchip Bump
INFO 04:07PM [connect_design]: 	Opened active design
	Design:    Planar_Final3 [Solution type: Eigenmode]
INFO 04:07PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 04:07PM [connect]: 	Connected to project "KPS_Flipchip Bump" and design "Planar_Final3" 😀 

INFO 04:07PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 04:07PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 04:07PM [analyze]: Analyzing setup Setup
INFO 04:09PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmphjkd9_s_.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mS

,ground_main_plane,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground_main_plane,157.180,-39.3120,-33.702,-26.9240
pad_bot_Q0,-39.312,82.1050,-36.754,-2.8344
pad_top_Q0,-33.702,-36.7540,93.032,-19.8480
quad_coupler_c0,-26.924,-2.8344,-19.848,51.0650


In [63]:
c1.setup.junctions = Dict({'Lj': 12,'Cj': 2})
c1.setup.freq_readout = 7.00
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

INFO 04:09PM [__del__]: Disconnected from Ansys HFSS
INFO 04:09PM [__del__]: Disconnected from Ansys HFSS
INFO 04:09PM [__del__]: Disconnected from Ansys HFSS


[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.488806 [GHz]
EC 310.727193 [MHz]
EJ 13.616300 [GHz]
alpha -361.275078 [MHz]
dispersion 36.676438 [KHz]
Lq 11.995161 [nH]
Cq 62.338374 [fF]
T1 54.406839 [us]

**Coupling Properties**

tCqbus1 -7.155333 [fF]
gbus1_in_MHz -111.139258 [MHz]
χ_bus1 -3.212974 [MHz]
1/T1bus1 2925.274581 [Hz]
T1bus1 54.406839 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.895078,362.154988,13.6163,-427.893368,138.063873,[-85.51695751006702],[-3.7342197064828846],3.734220,85.516958
2,5.877857,359.882014,13.6163,-424.90759,131.015574,[-104.45262530564528],[-5.3986633142110865],5.398663,104.452625
3,5.796115,349.20678,13.6163,-410.936506,101.724384,[-103.07416171820387],[-4.5466323494986325],4.546632,103.074162
4,5.693767,336.10275,13.6163,-393.902726,73.330474,[-108.81013218400939],[-4.25977919668607],4.259779,108.810132
5,5.60788,325.329576,13.6163,-379.993453,55.198078,[-108.90652952042984],[-3.7122779108188353],3.712278,108.906530
6,5.564683,319.987593,13.6163,-373.127803,47.688402,[-109.92921895509564],[-3.5331208799408955],3.533121,109.929219
7,5.528434,315.54398,13.6163,-367.432529,42.105162,[-109.97606316040506],[-3.34263410978573],3.342634,109.976063
8,5.503867,312.552795,13.6163,-363.606833,38.66108,[-110.65120997040441],[-3.2586675261780256],3.258668,110.651210
9,5.488806,310.727193,13.6163,-361.275078,36.676438,[-111.13925770670205],[-3.212973776197271],3.212974,111.139258


In [54]:
c1.sim.close()